In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re

In [ ]:
articles = pd.read_csv('/content/drive/MyDrive/datasets/UpdatedResumeDataSet.csv')

In [ ]:
articles.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [ ]:
import nltk
nltk.download('punkt')
data = list(articles['Resume'])
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model = Doc2Vec(vector_size = 50,
min_count = 10,
epochs = 50
)

In [ ]:
model.build_vocab(tagged_data)
k = model.wv.key_to_index.keys()
print(len(k))

4661


In [ ]:
model.train(tagged_data, total_examples = model.corpus_count, epochs = model.epochs)
model.save('/content/drive/MyDrive/resumeScreeningApp/doc2vec.model')
print("Model saved")

Model saved


In [27]:
resume_path = '/content/drive/MyDrive/functionalsample.pdf'
resume = ''
pdfReader_ = PyPDF2.PdfReader(resume_path)
for page in pdfReader_.pages:
    pageObj = page
    resume += pageObj.extract_text()

In [28]:
resume = resume.lower()
resume = re.sub('[^a-z]', ' ', resume)

In [29]:
def extract_data(url):
  list1 = []
  count = 0
  resp = requests.get(url)
  if resp.status_code == 200:
    soup = BeautifulSoup(resp.text,'html.parser')
    l = soup.find(class_ = 'av-company-description-page mb-2')
    web = ''.join([i.text for i in l.find_all(['p', 'li'])])
    list1.append(web)
    return web
  else:
    print("Error")

In [32]:
jd_links = ['https://datahack.analyticsvidhya.com/jobathon/clix-capital/senior-manager-growthrisk-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-growth-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-risk-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/cropin/data-scientist-85']

jd_df = pd.DataFrame(columns = ['links', 'data'])
jd_df['links'] = jd_links

# Extracting the JD data from the links:
for i in range(len(jd_df)):
  jd_df['data'][i] = extract_data(jd_df['links'][i])

In [33]:
#Converting the text into lower case
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : str.lower(x))

#Removing the punctuations from the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : " ".join(re.findall('[\w]+',x))
)
#Removing the numerics present in the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : re.sub(r'\d+','',x))

In [36]:
import numpy as np

model = Doc2Vec.load('doc2vec.model')
v1 = model.infer_vector(resume.split())
v2 = model.infer_vector(jd_df['data'][0].split())
cosine_similarity = (np.dot(np.array(v1), np.array(v2))) / (np.linalg.norm(np.array(v1)) * np.linalg.norm(np.array(v2)))
print(round(cosine_similarity, 3))


0.22
